In [ ]:
import cupy as cp
import cupyx
import scanpy as sc
import spatialleiden as sl
import squidpy as sq
import numpy as np
from cupyx.scipy.sparse import csr_matrix
import os
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import random
import pandas as pd
# from rsc_functions.utility.applyqc import applyqc
# from rsc_functions.reports.plot import plot_spatial,plot_spatial_data, plot_dist
# from rsc_functions.utility.rank_genes_groups import return_markers,rank_genes_groups
# from rsc_functions.reports.plot import plot_expression

<center><H2>Visium HD colon example </center>

In [ ]:
path_016 = "/data/kanferg/Sptial_Omics/playGround/Data/Visium_HD_Human_Colon_Cancer_binned_outputs/binned_outputs/square_016um"
pathout = "/data/kanferg/Sptial_Omics/SpatialOmicsToolkit/out_2"
andata = sc.read_h5ad(os.path.join(pathout, "andata_save_colon.h5ad"))
def load_matrix(andata,pathout,npz_file = "obsp_distances.npz",mode = 'sparse', mat_name = 'distances'):
    npzfile = cp.load(os.path.join(pathout, npz_file))
    data = cp.array(npzfile['data'])
    indices = cp.array(npzfile['indices'])
    indptr = cp.array(npzfile['indptr'])
    shape = tuple(npzfile['shape'])
    
    # Reconstruct the sparse matrix
    sparse_matrix_distances = csr_matrix((data, indices, indptr), shape=shape)
    if mode== 'sparse':
        andata.obsp[mat_name] = sparse_matrix_distances
    else:
        andata.obsp[mat_name] = sparse_matrix_distances.get()
    return andata

In [ ]:
andata.obs["imagerow"] = andata.obsm['spatial'][:,0]
andata.obs["imagecol"] = andata.obsm['spatial'][:,1]
andata.uns["spatial"] = andata.obsm["spatial"]
def make_uns_spatial(adata):
    #max_size = np.max([adata.obs["imagerow"].max(), adata.obs["imagecol"].max()])
    #max_size = int(max_size + 0.1 * max_size)
    #image = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 255))
    #imgarr = np.array(image)
    adata.uns["spatial"] = {}
    adata.uns["spatial"]["id1"] = {}
    #adata.uns["spatial"]["id1"]["images"] = {}
    #adata.uns["spatial"]["id1"]["images"]["lowres"] = imgarr
    adata.uns["spatial"]["id1"]["use_quality"] = "lowres"
    adata.uns["spatial"]["id1"]["scalefactors"] = {}
    adata.uns["spatial"]["id1"]["scalefactors"]["tissue_low_scalef" ] = 1
    adata.uns["spatial"]["id1"]["scalefactors"]["spot_diameter_fullres"] = 15
    adata.uns["spatial"]["id1"]["scalefactors"][
            "tissue_" + "lowres" + "_scalef"
        ] = 1
    return adata
andata = make_uns_spatial(andata)

In [ ]:
andata = load_matrix(andata,pathout,npz_file = "obsp_distances_large_colon.npz", mat_name = 'distances',mode="scipy")
andata = load_matrix(andata,pathout,npz_file = "obsp_connectivities_large_colon.npz", mat_name = 'connectivities',mode="scipy")

In [ ]:
andata

In [ ]:
sq.gr.spatial_neighbors(andata, coord_type="generic")

In [ ]:
andata

In [ ]:
andata.obsp["spatial_connectivities"] = sl.distance2connectivity(andata.obsp["spatial_distances"])

In [ ]:
seed = 42
sl.spatialleiden(andata, layer_ratio=1.5, directed=(False, True), seed=seed)

In [ ]:
andata

In [ ]:
sc.pl.embedding(andata, basis="spatial", color=["spatialleiden"])

In [ ]:
andata

In [ ]:
import os
import pickle

file_path = os.path.join(pathout, "andata_uns_mtracies__colon.pkl")

if os.path.getsize(file_path) > 0:
    with open(file_path, 'rb') as buff:
        andata_uns_mtracies = pickle.load(buff)
else:
    print("File is empty. Cannot load data.")
    andata_uns_mtracies = None
andata.uns["spatial"] = andata.obsm["spatial"]
andata.uns['clusterColorMap'] = andata_uns_mtracies['clusterColorMap']
andata.obs["imagerow"] = andata.obsm['spatial'][:,0]
andata.obs["imagecol"] = andata.obsm['spatial'][:,1]
andata.uns["spatial"] = andata.obsm["spatial"]

In [ ]:
df = pd.DataFrame({'cluster':andata.obs['spatialleiden'],'x':andata.obsm['spatial'][:,0],'y':andata.obsm['spatial'][:,1]})
color_map = andata.uns['clusterColorMap'].set_index('clusters')['colors'].to_dict()
df['color'] = df['cluster'].astype(str).map(color_map)

In [ ]:
sns.set_theme(style="whitegrid", palette="pastel")
color_container = []

fig, ax = plt.subplots(1, 1, figsize=(15,15))
clusters = sorted(np.unique(df['cluster'].values), key=int)
for i, cluster in enumerate(clusters):
    cluster_data = df[df['cluster'] == cluster]
    color_temp = cluster_data['color'][0]
    ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.1, alpha=1)

ax.set_xticks([])
ax.set_yticks([])
# ax.set_xlabel(xlab)
# ax.set_ylabel(ylab)
# ax.set_title(title)
legend = ax.legend(title='cluster',
                    bbox_to_anchor=(1.05, 1),  # Position the legend outside the plot
                    loc='upper left',
                    fontsize='small',  # Control the font size
                    title_fontsize='medium',
                     markerscale=30,  # Increase the size of the legend markers
                    frameon=False)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)

In [ ]:
range(14

In [ ]:
plt.rcParams['figure.dpi'] = 92
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
fig, ax = plt.subplots(7,2, figsize=(10, 50))
ax = ax.ravel()
for i in range(14):
    cluster_data = df[df['cluster'] == i]
    color_temp = cluster_data['color'][0]
    ax[i].scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label='Goblet cells', s=0.5, alpha=1)
    ax[i].set_title(str(i))
plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.2)

In [ ]:
plt.rcParams['figure.dpi'] = 92
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
fig, ax = plt.subplots(1,1, figsize=(7,7))
cluster_data = df[df['cluster'] == 14]
color_temp = cluster_data['color'][0]
ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label='Goblet cells', s=0.5, alpha=1)
ax.set_title(str(14))
plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.2)

In [ ]:
cluster = np.arange(15).tolist()
spatialleiden_seq = [0,1,7,4,10,2,13,11,8,6,10,5,12,3,14]


In [ ]:
spatialleiden_map = {spatialleiden_seq.pop(0):i for i in np.arange(15).tolist()}

In [ ]:
andata.obs['spatialleiden_mapped'] = andata.obs['spatialleiden'].map(spatialleiden_map)

In [ ]:
andata.obs['spatialleiden_mapped']

In [ ]:
df = pd.DataFrame({'cluster':andata.obs['spatialleiden_mapped'],'x':andata.obsm['spatial'][:,0],'y':andata.obsm['spatial'][:,1]})
df = df.dropna(subset=['cluster'])
df['cluster'] = df['cluster'].astype(int)
color_map = andata.uns['clusterColorMap'].set_index('clusters')['colors'].to_dict()
df['color'] = df['cluster'].astype(str).map(color_map)

In [ ]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 800
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

color_container = []
fig, ax = plt.subplots(1, 1, figsize=(15,15))
clusters = sorted(np.unique(df['cluster'].values), key=int)
for i, cluster in enumerate(clusters):
    cluster_data = df[df['cluster'] == cluster]
    color_temp = cluster_data['color'][0]
    ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.3, alpha=1)

ax.set_xticks([])
ax.set_yticks([])
# ax.set_xlabel(xlab)
# ax.set_ylabel(ylab)
# ax.set_title(title)
legend = ax.legend(title='cluster',
                    bbox_to_anchor=(1.05, 1),  # Position the legend outside the plot
                    loc='upper left',
                    fontsize='small',  # Control the font size
                    title_fontsize='medium',
                     markerscale=20,  # Increase the size of the legend markers
                    frameon=False)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)

In [ ]:
with PdfPages(os.path.join(pathout, f'Report_spatial_map_plot_colon_spatialleiden.pdf')) as pdf:
    plt.rcParams['figure.dpi'] = 600
    plt.rcParams['font.family'] = ['serif']
    plt.rcParams['font.size'] = 12
    plt.rcParams['axes.labelsize'] = 12
    plt.rcParams['axes.titlesize'] = 12
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    fig, ax = plt.subplots(1,1, figsize=(15, 15))
    color_container = []
    clusters = sorted(np.unique(df['cluster'].values), key=int)
    for i, cluster in enumerate(clusters):
        cluster_data = df[df['cluster'] == cluster]
        color_temp = cluster_data['color'][0]
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.5, alpha=1)
    
    ax.set_xticks([])
    ax.set_yticks([])
    # ax.set_xlabel(xlab)
    # ax.set_ylabel(ylab)
    # ax.set_title(title)
    legend = ax.legend(title='cluster',
                        bbox_to_anchor=(1.05, 1),  # Position the legend outside the plot
                        loc='upper left',
                        fontsize='small',  # Control the font size
                        title_fontsize='medium',
                         markerscale=10,  # Increase the size of the legend markers
                        frameon=False)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)
    pdf.savefig()
    plt.close()

In [ ]:
df[df['cluster'] == cluster]

In [ ]:
sorted(np.unique(df['cluster'].values), key=int)

In [ ]:
df = pd.DataFrame({'cluster':andata.obs['spatialleiden_mapped'],'x':andata.obsm['spatial'][:,0],'y':andata.obsm['spatial'][:,1]})
df = df.dropna(subset=['cluster'])
df['cluster'] = df['cluster'].astype(int)
color_map = andata.uns['clusterColorMap'].set_index('clusters')['colors'].to_dict()
df['color'] = df['cluster'].astype(str).map(color_map)

In [ ]:
plt.rcParams['figure.dpi'] = 600
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
fig, ax = plt.subplots(1,1, figsize=(15, 15))
clusters = sorted(np.unique(df['cluster'].values), key=int)
for i, cluster in enumerate(clusters):
    cluster_data = df[df['cluster'] == cluster]
    color_temp = cluster_data['color'][0]
    if i==6:
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label='Goblet cells', s=0.5, alpha=1)
    elif i == 10:
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label="Tumor", s=0.5, alpha=1)
    else:
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.5, alpha=0.05)
ax.set_xticks([])
ax.set_yticks([])
# ax.set_xlabel(xlab)
# ax.set_ylabel(ylab)
# ax.set_title(title)
legend = ax.legend(title='cluster',
                    bbox_to_anchor=(1.2, 1),  # Position the legend outside the plot
                    loc='upper left',
                    fontsize='small',  # Control the font size
                    title_fontsize='medium',
                     markerscale=10,  # Increase the size of the legend markers
                    frameon=False)
plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.2)

In [ ]:
with PdfPages(os.path.join(pathout, f'Report_spatial_map_plot_colon_celltype_tumer_goblet_Spatialleiden.pdf')) as pdf:
    plt.rcParams['figure.dpi'] = 600
    plt.rcParams['font.family'] = ['serif']
    plt.rcParams['font.size'] = 12
    plt.rcParams['axes.labelsize'] = 12
    plt.rcParams['axes.titlesize'] = 12
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    fig, ax = plt.subplots(1,1, figsize=(15, 15))
    clusters = sorted(np.unique(df['cluster'].values), key=int)
    for i, cluster in enumerate(clusters):
        cluster_data = df[df['cluster'] == cluster]
        color_temp = cluster_data['color'][0]
        if i==6:
            ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label='Goblet cells', s=0.5, alpha=1)
        elif i == 10:
            ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label="Tumor", s=0.5, alpha=1)
        else:
            ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.5, alpha=0.05)
    ax.set_xticks([])
    ax.set_yticks([])
    # ax.set_xlabel(xlab)
    # ax.set_ylabel(ylab)
    # ax.set_title(title)
    legend = ax.legend(title='cluster',
                        bbox_to_anchor=(1.2, 1),  # Position the legend outside the plot
                        loc='upper left',
                        fontsize='small',  # Control the font size
                        title_fontsize='medium',
                         markerscale=10,  # Increase the size of the legend markers
                        frameon=False)
    plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.2)
    pdf.savefig()
    plt.close()

<center><H2>multiSPAETI colon visium data</center>

In [ ]:
from multispaeti import MultispatiPCA
import cupy as cp
import cupyx
import scanpy as sc
import spatialleiden as sl
import squidpy as sq
import numpy as np
from cupyx.scipy.sparse import csr_matrix
import os
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import random
import pandas as pd

In [ ]:
path_016 = "/data/kanferg/Sptial_Omics/playGround/Data/Visium_HD_Human_Colon_Cancer_binned_outputs/binned_outputs/square_016um"
pathout = "/data/kanferg/Sptial_Omics/SpatialOmicsToolkit/out_2"
andata = sc.read_h5ad(os.path.join(pathout, "andata_save_colon.h5ad"))
def load_matrix(andata,pathout,npz_file = "obsp_distances.npz",mode = 'sparse', mat_name = 'distances'):
    npzfile = cp.load(os.path.join(pathout, npz_file))
    data = cp.array(npzfile['data'])
    indices = cp.array(npzfile['indices'])
    indptr = cp.array(npzfile['indptr'])
    shape = tuple(npzfile['shape'])
    
    # Reconstruct the sparse matrix
    sparse_matrix_distances = csr_matrix((data, indices, indptr), shape=shape)
    if mode== 'sparse':
        andata.obsp[mat_name] = sparse_matrix_distances
    else:
        andata.obsp[mat_name] = sparse_matrix_distances.get()
    return andata

In [ ]:
andata.obs["imagerow"] = andata.obsm['spatial'][:,0]
andata.obs["imagecol"] = andata.obsm['spatial'][:,1]
andata.uns["spatial"] = andata.obsm["spatial"]
def make_uns_spatial(adata):
    #max_size = np.max([adata.obs["imagerow"].max(), adata.obs["imagecol"].max()])
    #max_size = int(max_size + 0.1 * max_size)
    #image = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 255))
    #imgarr = np.array(image)
    adata.uns["spatial"] = {}
    adata.uns["spatial"]["id1"] = {}
    #adata.uns["spatial"]["id1"]["images"] = {}
    #adata.uns["spatial"]["id1"]["images"]["lowres"] = imgarr
    adata.uns["spatial"]["id1"]["use_quality"] = "lowres"
    adata.uns["spatial"]["id1"]["scalefactors"] = {}
    adata.uns["spatial"]["id1"]["scalefactors"]["tissue_low_scalef" ] = 1
    adata.uns["spatial"]["id1"]["scalefactors"]["spot_diameter_fullres"] = 15
    adata.uns["spatial"]["id1"]["scalefactors"][
            "tissue_" + "lowres" + "_scalef"
        ] = 1
    return adata
andata = make_uns_spatial(andata)

andata = load_matrix(andata,pathout,npz_file = "obsp_distances_large_colon.npz", mat_name = 'distances',mode="scipy")
andata = load_matrix(andata,pathout,npz_file = "obsp_connectivities_large_colon.npz", mat_name = 'connectivities',mode="scipy")

sq.gr.spatial_neighbors(andata, coord_type="generic")
andata.obsp["spatial_connectivities"] = sl.distance2connectivity(andata.obsp["spatial_distances"])

In [ ]:
import os
import pickle

file_path = os.path.join(pathout, "andata_uns_mtracies__colon.pkl")

if os.path.getsize(file_path) > 0:
    with open(file_path, 'rb') as buff:
        andata_uns_mtracies = pickle.load(buff)
else:
    print("File is empty. Cannot load data.")
    andata_uns_mtracies = None
andata.uns["spatial"] = andata.obsm["spatial"]
andata.uns['clusterColorMap'] = andata_uns_mtracies['clusterColorMap']
andata.obs["imagerow"] = andata.obsm['spatial'][:,0]
andata.obs["imagecol"] = andata.obsm['spatial'][:,1]
andata.uns["spatial"] = andata.obsm["spatial"]

In [ ]:
connectivity = andata.obsp["spatial_connectivities"]

In [ ]:
msPCA = MultispatiPCA(n_components=(30, 5), connectivity=connectivity)

In [ ]:
msPCA.fit(andata.X)

In [ ]:
X_transformed = msPCA.transform(andata.X)

In [ ]:
andata.obsm['X_pca'].shape

In [ ]:
andata.obsm['X_pca_spat'] = X_transformed

In [ ]:
sc.pp.neighbors(andata, n_pcs=15, use_rep='X_pca_spat', n_neighbors=25,key_added = 'pca_spat')
andata

In [ ]:
sc.tl.leiden(andata, random_state=1337, resolution=0.5, key_added='leiden',neighbors_key = 'pca_spat')

In [ ]:
andata

In [ ]:
df = pd.DataFrame({'cluster':andata.obs['leiden'],'x':andata.obsm['spatial'][:,0],'y':andata.obsm['spatial'][:,1]})
color_map = andata.uns['clusterColorMap'].set_index('clusters')['colors'].to_dict()
df['color'] = df['cluster'].astype(str).map(color_map)

In [ ]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 800
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

color_container = []
fig, ax = plt.subplots(1, 1, figsize=(15,15))
clusters = sorted(np.unique(df['cluster'].values), key=int)
for i, cluster in enumerate(clusters):
    cluster_data = df[df['cluster'] == cluster]
    color_temp = cluster_data['color'][0]
    ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.3, alpha=1)

ax.set_xticks([])
ax.set_yticks([])
# ax.set_xlabel(xlab)
# ax.set_ylabel(ylab)
# ax.set_title(title)
legend = ax.legend(title='cluster',
                    bbox_to_anchor=(1.05, 1),  # Position the legend outside the plot
                    loc='upper left',
                    fontsize='small',  # Control the font size
                    title_fontsize='medium',
                     markerscale=20,  # Increase the size of the legend markers
                    frameon=False)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)

In [ ]:
color_map['11']

In [ ]:
color_temp

In [ ]:
plt.rcParams['figure.dpi'] = 600
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
fig, ax = plt.subplots(1,1, figsize=(15, 15))
clusters = sorted(np.unique(df['cluster'].values), key=int)
for i, cluster in enumerate(clusters):
    cluster_data = df[df['cluster'] == cluster]
    color_temp = cluster_data['color'][0]
    if i==4:
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_map['7'], label='Goblet cells', s=0.5, alpha=1)
    elif i == 5:
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_map['11'], label="Tumor", s=0.5, alpha=1)
    else:
        ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.5, alpha=0.05)
ax.set_xticks([])
ax.set_yticks([])
# ax.set_xlabel(xlab)
# ax.set_ylabel(ylab)
# ax.set_title(title)
legend = ax.legend(title='cluster',
                    bbox_to_anchor=(1.2, 1),  # Position the legend outside the plot
                    loc='upper left',
                    fontsize='small',  # Control the font size
                    title_fontsize='medium',
                     markerscale=10,  # Increase the size of the legend markers
                    frameon=False)
plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.2)

In [ ]:
with PdfPages(os.path.join(pathout, f'Report_spatial_map_plot_colon_celltype_tumer_goblet_SpatPCA.pdf')) as pdf:
    plt.rcParams['figure.dpi'] = 600
    plt.rcParams['font.family'] = ['serif']
    plt.rcParams['font.size'] = 12
    plt.rcParams['axes.labelsize'] = 12
    plt.rcParams['axes.titlesize'] = 12
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    fig, ax = plt.subplots(1,1, figsize=(15, 15))
    clusters = sorted(np.unique(df['cluster'].values), key=int)
    for i, cluster in enumerate(clusters):
        cluster_data = df[df['cluster'] == cluster]
        color_temp = cluster_data['color'][0]
        if i==4:
            ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_map['7'], label='Goblet cells', s=0.5, alpha=1)
        elif i == 5:
            ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_map['11'], label="Tumor", s=0.5, alpha=1)
        else:
            ax.scatter( x=cluster_data['x'], y=cluster_data['y'], color=color_temp, label=f'{cluster}', s=0.5, alpha=0.05)
    ax.set_xticks([])
    ax.set_yticks([])
    # ax.set_xlabel(xlab)
    # ax.set_ylabel(ylab)
    # ax.set_title(title)
    legend = ax.legend(title='cluster',
                        bbox_to_anchor=(1.2, 1),  # Position the legend outside the plot
                        loc='upper left',
                        fontsize='small',  # Control the font size
                        title_fontsize='medium',
                         markerscale=10,  # Increase the size of the legend markers
                        frameon=False)
    plt.subplots_adjust(left=0.1, right=0.75, top=0.9, bottom=0.2)
    pdf.savefig()
    plt.close()

<center><H2>Xinium 5k lymph nodes example </center>

In [ ]:
path = "/data/kanferg/Sptial_Omics/playGround/Data/Xenium/output_temp"
pathout = "/data/kanferg/Sptial_Omics/SpatialOmicsToolkit/out_1"
FilePrefix = "_072824" 
andata = sc.read_h5ad(os.path.join(pathout, "andata_save.h5ad"))
def load_matrix(andata,pathout,npz_file = "obsp_distances.npz",mode = 'sparse', mat_name = 'distances'):
    npzfile = cp.load(os.path.join(pathout, npz_file))
    data = cp.array(npzfile['data'])
    indices = cp.array(npzfile['indices'])
    indptr = cp.array(npzfile['indptr'])
    shape = tuple(npzfile['shape'])
    
    # Reconstruct the sparse matrix
    sparse_matrix_distances = csr_matrix((data, indices, indptr), shape=shape)
    if mode== 'sparse':
        andata.obsp[mat_name] = sparse_matrix_distances
    else:
        andata.obsp[mat_name] = sparse_matrix_distances.get()
    return andata

In [ ]:
andata = load_matrix(andata,pathout,npz_file = "obsp_distances_large.npz", mat_name = 'distances',mode="scipy")
andata = load_matrix(andata,pathout,npz_file ="obsp_connectivities_large.npz", mat_name = 'connectivities',mode="scipy")

In [ ]:
andata.obs["imagerow"] = andata.obs["cell_centroid_x"]
andata.obs["imagecol"] = andata.obs["cell_centroid_y"]
andata.uns["spatial"] = andata.obsm["spatial"]
def make_uns_spatial(adata):
    max_size = np.max([adata.obs["imagerow"].max(), adata.obs["imagecol"].max()])
    max_size = int(max_size + 0.1 * max_size)
    image = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 255))
    imgarr = np.array(image)
    adata.uns["spatial"] = {}
    adata.uns["spatial"]["id1"] = {}
    adata.uns["spatial"]["id1"]["images"] = {}
    adata.uns["spatial"]["id1"]["images"]["lowres"] = imgarr
    adata.uns["spatial"]["id1"]["use_quality"] = "lowres"
    adata.uns["spatial"]["id1"]["scalefactors"] = {}
    adata.uns["spatial"]["id1"]["scalefactors"]["tissue_low_scalef" ] = 1
    adata.uns["spatial"]["id1"]["scalefactors"]["spot_diameter_fullres"] = 15
    adata.uns["spatial"]["id1"]["scalefactors"][
            "tissue_" + "lowres" + "_scalef"
        ] = 1
    return adata
andata = make_uns_spatial(andata)

In [ ]:
import os
import pickle

file_path = os.path.join(pathout, "andata_uns_mtracies.pkl")

if os.path.getsize(file_path) > 0:
    with open(file_path, 'rb') as buff:
        andata_uns_mtracies = pickle.load(buff)
else:
    print("File is empty. Cannot load data.")
    andata_uns_mtracies = None

In [ ]:
andata.uns['cluster_colors'] = andata_uns_mtracies['cluster_colors']
andata.uns['clusterColorMap'] = andata_uns_mtracies['clusterColorMap']

In [ ]:
andata

In [ ]:
from scipy.sparse import csr_matrix
andata_sub = andata.copy()
andata_sub.X = csr_matrix(andata_sub.X)
andata_sub = sc.pp.subsample(andata_sub, n_obs=100000,copy=True)

In [ ]:
# sort colors by cluster from zero to 15
andata.uns['clusterColorMap'].index = andata.uns['clusterColorMap']['clusters']
# creat a cluster list 
clusterColorMap = andata.uns['clusterColorMap'].sort_index()['colors'].to_list()

In [ ]:
df = pd.DataFrame({'cluster':andata.obs['cluster'],'x':andata.obsm['spatial'][:,1],'y':andata.obsm['spatial'][:,0]}).reset_index()

In [ ]:
# colors saved from rsc analysis step
palette = andata.uns['clusterColorMap'].sort_index()['colors'].to_list()
clusters = sorted(np.unique(df['cluster'].values), key=int)
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 800
plt.rcParams['font.family'] = ['serif']
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
fig, axes = plt.subplots(1, 1, figsize=(15, 15))
for i, cluster in enumerate(clusters):
    cluster_grid = df[df['cluster'] == cluster]
    axes.scatter(x=cluster_grid['x'], y=cluster_grid['y'], color=palette[i], label=f'{cluster}', s=0.02, alpha=1)
axes.set_xticks([])
axes.set_yticks([])
axes.set_xlabel('')
axes.set_ylabel('')
axes.set_title(f'Grid cluster dominant spots')
legend1 = axes.legend( title="Cluster",
                        bbox_to_anchor=(1.05, 1),  # Position the legend outside the plot
                        loc='upper left',
                        fontsize='small',  # Control the font size
                        title_fontsize='medium',
                        markerscale=20,  # Increase the size of the legend markers
                        frameon=False# Control the title font size
                        )
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)

In [ ]:
andata_sub

In [ ]:
sq.gr.spatial_neighbors(andata_sub, coord_type="generic",delaunay = True)

In [ ]:
andata_sub

In [ ]:
andata_sub.obsp["spatial_connectivities"] = sl.distance2connectivity(andata_sub.obsp["spatial_distances"])

In [ ]:
seed = 42
sl.spatialleiden(andata_sub, layer_ratio=0.2, directed=(False, True), seed=seed)

In [ ]:
andata_sub

In [ ]:
sc.pl.embedding(andata_sub, basis="spatial", color=["spatialleiden"])